# Just Bundle Things

This script runs through some things you can do with owmeta bundles

 - [BASICS](#BASICS) Creating and listing bundles
 - [FETCH](#FETCH) Getting bundles
 - [LOCAL](#LOCAL) Local index and local cache
 - [REMOTE](#REMOTE) Remote repositories and distributed file shares
 - [INSTALL](#INSTALL) Installing a bundle to the local index

We'll do everything in a temporary directory and clean up at the end

In [ ]:
# You can re-run this cell to restart from a fresh directory
START_DIRECTORY=${START_DIRECTORY:-$(pwd)}
cd "$START_DIRECTORY"
if type cleanup > /dev/null 2> /dev/null ; then
    # Clean up if we've run this cell before
    cleanup
fi
# Make a working directory
working_directory=$(mktemp -d -t jbt.XXXXXXXXXX)
# Change the home directory -- some `owm` commands make use of it,
# but we don't want to mess up our actual home directory
export HOME="$working_directory/home"
# Copy over some files we'll need later
mkdir "$working_directory/scripts"
cp "$START_DIRECTORY/fake_bundle_server.py" "$working_directory/scripts/"

cd $working_directory
cleanup () {
    cd /
    if [ -d "$working_directory" ] ; then
        rm -rf "$working_directory"
    fi
}
# Clean up when the script exits normally or with an "INT" signal
trap cleanup EXIT INT

In [ ]:
echo $(pwd)

We'll also be using the common owmeta database

In [ ]:
rm -rf .owm
owm clone https://github.com/openworm/OpenWormData.git --branch=461-pow-om-rename

## BASICS

To create a bundle, you have to register a bundle descriptor. This is a file which describes what's in a bundle.

You can make a bundle descriptor file with any editor you like. It's a YAML serialization.

In [ ]:
cat > bundle.yml << HERE
---
id: example/abundle
description: |-
    This is a bundle used for demonstration purposes
version: 1
includes:
    - http://openworm.org/schema/sci
patterns:
    - rgx:.*data_sources.*#Neurons/context_for.*
    - '*/translators/*'
files:
    includes:
        - src/readme.txt
    patterns:
        - 'src/data_set_0[0-9][0-9]/**'
HERE

The `owm bundle register` command actually registers the bundle. This puts a reference to the descriptor file in the .owm directory


In [ ]:
owm bundle register bundle.yml

`owm bundle list` lists the bundles registered in this owmeta project

In [ ]:
owm bundle list

If you move or rename a bundle file owmeta will not know about it: it does not track file moves. It will, however, tell you if a bundle descriptor cannot be found when you list registered bundles.

In [ ]:
mv bundle.yml aBundle.yml
owm bundle list

To correct this, you must re-register the bundle at the new location

In [ ]:
owm bundle register aBundle.yml
owm bundle list

If you're done with a bundle, you deregister it. You can provide either the descriptor file name or the bundle ID

In [ ]:
owm bundle deregister 'example/abundle'
owm bundle list

## FETCH

[BASIC](#BASIC) showed us how to create and list bundles, but what about when you want someone else's bundles? To fetch a bundle, you need the bundle name. That name is queried for in your local repository (see below in [LOCAL](#LOCAL)), then in any remotes configured in your ".owm" directory or user settings, and finally the default remote (see [REMOTE](#REMOTE)). `owm bundle fetch` does this for you.

For the purposes of this example, let's start up a local bundle server and add a remote for it:

In [ ]:
if [ "$SERVER_PID" ] ; then pkill -P $SERVER_PID python ; fi
if [ -f server_address ] ; then rm server_address ; fi
python scripts/fake_bundle_server.py server_address > fake_bundle_server.stdout.log 2> fake_bundle_server.stderr.log &
SERVER_PID=$!

In [ ]:
server_url=$(cat server_address)
while [ ! "$server_url" ] ; do
    server_url=$(cat server_address)
    echo "Waiting..."
    sleep 1
done
echo $server_url

In [ ]:
cat fake_bundle_server.stderr.log

In [ ]:
owm bundle remote add example $server_url/bundles.idx
owm bundle remote list

In [ ]:
owm bundle fetch example/aBundle

Fetching a bundle puts it in your local repository so you can use it in any projects on the local machine. You use a bundle in Python with the Bundle object. You can access contexts within a bundle by passing the context to the bundle as shown below.

In [ ]:
use_a_bundle() {
    BUNDLE="$1"
    CONTEXT="$2"
    cat > scripts/use_a_bundle.py << HERE
'''
List DataObject sub-classes in the bundle

This is a way to learn a little about the classes in a bundle
'''
from owmeta.bundle import Bundle
from owmeta.context import Context
from owmeta.dataObject import (TypeDataObject,
                               DataObject,
                               RDFSSubClassOfProperty,
                               BaseDataObject)

# Use the bundle
with Bundle('${BUNDLE}') as bnd:
    # "contextualize" the Context with the bundle to access contexts within the bundle
    ctx = bnd(Context)('${CONTEXT}')
    for x in ctx.stored.rdf_graph():
        print(x)
    # Make a query for subclasses of BaseDataObject -- the owmeta type which represents
    # RDF resources
    tdo = ctx.stored(TypeDataObject).query()
    tdo.attach_property(RDFSSubClassOfProperty)
    tdo.rdfs_subclassof_property(BaseDataObject.rdf_type_object)
    # Execute the query
    for sc in tdo.load():
        print(sc)
HERE

    python scripts/use_a_bundle.py
}


In [ ]:
use_a_bundle 'example/bundle.01' 'https://example.org/bundles#example'

Note that the bundle does not need to have been already fetched. In use_a_bundle.py, we use a bundle `example/bundle.01` which we had not previously fetched. When you make the Bundle object, owmeta will retrieve the bundle from remotes if necessary.